## Test Data Prediction
This notebook will be used to open test.csv file and use designated model to predict the test data for kaggle submission.

In [7]:
%%bash
git checkout 'feat2/wip'

error: Your local changes to the following files would be overwritten by checkout:
	Test_data.ipynb
Please commit your changes or stash them before you switch branches.
Aborting


In [8]:
%%bash
git add 'Test_data.ipynb'
git commit -m 'Branched to feat2'
git push --set-upstream origin feat2/wip

[master 499bd47] Preparing to branch to feat2 branch
 1 file changed, 20 insertions(+), 11 deletions(-)
Branch master set up to track remote branch master from origin.


To github.com:Theerit/Kaggle_Titanic.git
   9a1a201..499bd47  master -> master


In [2]:
model_ver = 'feat2.sav'

In [3]:
%matplotlib inline 
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn.model_selection
import pickle

/Users/theerit/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Load data from csv file then prepare the data before inputting into the trained model.

In [4]:
test_ex = pd.read_csv('gender_submission.csv')
test_ex.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [5]:
test = pd.read_csv('test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
test = test.drop('PassengerId',axis=1)
test = test.drop('Name',axis=1)

#To flag num attri features
num_attri = ['Age','SibSp','Parch','Fare']
cat_attri = [column for column in test if (column not in num_attri and column != 'Survived')]

#Creating dummy variable
for column in (cat_attri):
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(test[column],prefix=column)
    test.drop(column,axis=1,inplace=True)
    test = test.join(tempDF)
test.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Ticket_110469,...,Cabin_F E46,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Embarked_C,Embarked_Q,Embarked_S
0,34.5,0,0,7.8292,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,47.0,1,0,7.0000,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,62.0,0,0,9.6875,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,27.0,0,0,8.6625,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,22.0,1,1,12.2875,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
loaded_model = pickle.load(open(model_ver, 'rb'))
loaded_model.predict(test)

ValueError: feature_names mismatch: [u'Age', u'SibSp', u'Parch', u'Fare', u'Pclass_1', u'Pclass_2', u'Pclass_3', u'Sex_female', u'Sex_male', u'Ticket_110152', u'Ticket_110413', u'Ticket_110465', u'Ticket_110564', u'Ticket_110813', u'Ticket_111240', u'Ticket_111320', u'Ticket_111361', u'Ticket_111369', u'Ticket_111426', u'Ticket_111427', u'Ticket_111428', u'Ticket_112050', u'Ticket_112052', u'Ticket_112053', u'Ticket_112058', u'Ticket_112059', u'Ticket_112277', u'Ticket_112379', u'Ticket_113028', u'Ticket_113043', u'Ticket_113050', u'Ticket_113051', u'Ticket_113055', u'Ticket_113056', u'Ticket_113059', u'Ticket_113501', u'Ticket_113503', u'Ticket_113505', u'Ticket_113509', u'Ticket_113510', u'Ticket_113514', u'Ticket_113572', u'Ticket_113760', u'Ticket_113767', u'Ticket_113773', u'Ticket_113776', u'Ticket_113781', u'Ticket_113783', u'Ticket_113784', u'Ticket_113786', u'Ticket_113787', u'Ticket_113788', u'Ticket_113789', u'Ticket_113792', u'Ticket_113794', u'Ticket_113796', u'Ticket_1137

In [28]:
# load the model from disk
loaded_model = pickle.load(open(model_ver, 'rb'))


accuracy_score(y_test,clf2.predict(X_test))
result = loaded_model.score(X_test, Y_test)
print(result)

NameError: name 'y_test' is not defined